# Elastic Stackの構築手順：収容（GCE）

----

Elasticsearchのインストール先となるGoogle Compute&nbsp;Engineインスタンスを確保します。  
大まかな手順は次の通りです。  
1. 動作環境の確認
2. GCEインスタンスの設定を定義
3. GCEインスタンスの生成
4. Ansibleとの連携確認
5. Inventoryへのホスト追加

# 動作環境の確認

このNotebookは、 [Google Python Client Library](https://github.com/google/google-api-python-client) を使ってマシンの確保を行います。そのため、Libraryがインポート可能であることの確認と、Credentialsが設定されている

## Libraryの確認

このNotebook環境にGoogle Python Client Libraryがインストールされている必要があります。インストールされていない場合は、以下のセル実行に失敗し、 ImportError となります。

In [1]:
from googleapiclient import discovery

/usr/local/lib/python2.7/dist-packages/cffi/model.py:525: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


もし、インストールされていない場合は、[Using the Python Client Library](https://cloud.google.com/compute/docs/tutorials/python-guide)を参考にインストールしてください。

In [2]:
#!pip2 install --upgrade google-api-python-client
#from googleapiclient import discovery

## Credentialの指定

Google Compute EngineにアクセスするためのCredentialを指定してください。

- JSONフォーマットのService Account情報
- プロジェクトID

を用意しておく。

In [3]:
creds = '~/.keys/testdev-98ad0186d51d.json'
target_project_id = 'testdev'

computeサービスを取得しておく。

In [4]:
import os
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.from_stream(os.path.expanduser(creds))
compute = discovery.build('compute', 'v1', credentials=credentials)
compute

# GCEインスタンスの設定と生成

サーバ構成に応じたGCEインスタンスの設定を定義し、インスタンスを生成します。

## ゾーンの設定

どのZoneにインスタンスを確保するかを定義しておく。

In [5]:
target_zone = 'us-central1-f'

動作確認として、現在のインスタンス名の一覧を確認する。

In [7]:
instances = compute.instances().list(zone=target_zone, project=target_project_id).execute()
map(lambda i: i['name'], instances['items'] if 'items' in instances else [])

[]

## インスタンスの設定


### イメージの設定

マシンイメージを設定します。

まずイメージの一覧を確認する。projectには、利用したいイメージの所属プロジェクトを指定する。

In [8]:
images = compute.images().list(project='centos-cloud').execute()['items']
images = filter(lambda i: i['status'] == 'READY' and 'deprecated' not in i, images)
map(lambda i: (i['name'], i['selfLink']), images)

[(u'centos-6-v20170227',
  u'https://www.googleapis.com/compute/v1/projects/centos-cloud/global/images/centos-6-v20170227'),
 (u'centos-7-v20170227',
  u'https://www.googleapis.com/compute/v1/projects/centos-cloud/global/images/centos-7-v20170227')]

利用したいイメージのURLを設定する。

In [9]:
# centos-7-v20170227
source_disk_image = 'https://www.googleapis.com/compute/v1/projects/centos-cloud/global/images/centos-7-v20170227'

### マシンタイプの設定

[マシンタイプ](https://cloud.google.com/compute/docs/machine-types)のページから適切なタイプを選択してください。

例えば60GB程度のメモリを利用するのであれば **n1-standard-16** が選択候補となります。  
利用したいサイズに応じてタイプを選んで下さい。

このZoneで利用可能なMachine Typeは以下の通りです。

In [10]:
machineTypes = compute.machineTypes().list(project=target_project_id, zone=target_zone).execute()['items']
map(lambda t: (t['name'], t['description']), machineTypes)

[(u'f1-micro', u'1 vCPU (shared physical core) and 0.6 GB RAM'),
 (u'g1-small', u'1 vCPU (shared physical core) and 1.7 GB RAM'),
 (u'n1-highcpu-16', u'16 vCPUs, 14.4 GB RAM'),
 (u'n1-highcpu-2', u'2 vCPUs, 1.8 GB RAM'),
 (u'n1-highcpu-32', u'32 vCPUs, 28.8 GB RAM'),
 (u'n1-highcpu-4', u'4 vCPUs, 3.6 GB RAM'),
 (u'n1-highcpu-8', u'8 vCPUs, 7.2 GB RAM'),
 (u'n1-highmem-16', u'16 vCPUs, 104 GB RAM'),
 (u'n1-highmem-2', u'2 vCPUs, 13 GB RAM'),
 (u'n1-highmem-32', u'32 vCPUs, 208 GB RAM'),
 (u'n1-highmem-4', u'4 vCPUs, 26 GB RAM'),
 (u'n1-highmem-8', u'8 vCPUs, 52 GB RAM'),
 (u'n1-standard-1', u'1 vCPU, 3.75 GB RAM'),
 (u'n1-standard-16', u'16 vCPUs, 60 GB RAM'),
 (u'n1-standard-2', u'2 vCPUs, 7.5 GB RAM'),
 (u'n1-standard-32', u'32 vCPUs, 120 GB RAM'),
 (u'n1-standard-4', u'4 vCPUs, 15 GB RAM'),
 (u'n1-standard-8', u'8 vCPUs, 30 GB RAM')]

In [11]:
machine_type = 'n1-standard-16'

### キーペアの設定

現在のSSHキーの一覧を取得する。

In [12]:
projectMetadata = compute.projects().get(project=target_project_id).execute()
currentSSHKeys = filter(lambda metadata: metadata['key'] == 'sshKeys', projectMetadata['commonInstanceMetadata']['items']) \
                        if 'commonInstanceMetadata' in projectMetadata and 'items' in projectMetadata['commonInstanceMetadata'] else []
currentSSHKeys = currentSSHKeys[0]['value'].split('\n') if currentSSHKeys else []
currentSSHKeys

[u'',
 u'ansible:ssh-rsa XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ansible@XXXXXXXXXXXX']

SSHのキー一覧にこのNotebook環境のキーがなければ、追加する。

In [13]:
#pub_key = None
#with open(os.path.expanduser('~/.ssh/ansible_id_rsa.pub'), 'r') as f:
#    pub_key = f.readlines()[0].strip()
#
#if not filter(lambda k: k.endswith(pub_key), currentSSHKeys):
#    currentSSHKeys.append('ansible:' + pub_key)
#currentSSHKeys

Metadataに反映する。

In [14]:
#compute.projects().setCommonInstanceMetadata(project=target_project_id,
#                                             body={'items': [{'key': 'sshKeys',
#                                                              'value': '\n'.join(currentSSHKeys)}]}).execute()

Metadataに反映されたことを確認する。

In [15]:
#projectMetadata = compute.projects().get(project=target_project_id).execute()
#currentSSHKeys = filter(lambda metadata: metadata['key'] == 'sshKeys', projectMetadata['commonInstanceMetadata']['items']) \
#                        if 'commonInstanceMetadata' in projectMetadata and 'items' in projectMetadata['commonInstanceMetadata'] else []
#currentSSHKeys = currentSSHKeys[0]['value'].split('\n') if currentSSHKeys else []
#currentSSHKeys

### ディスクサイズの設定

ディスクに確保する容量を設定する。

In [16]:
disk_size_gb = 1024

### インスタンスの生成

設定した情報を用いてマシンを確保する。

作成するインスタンス数を設定します。

In [17]:
instance_count = 3

インスタンス群に付与する名前を設定します。  
この名前は、後の処理でタグのNameキーに設定されます。

In [18]:
instance_name = 'es-cluster-'

ディスクをインスタンス数だけ準備を行います。

In [19]:
new_disks = []
for i in range(0, instance_count):
    name = '{0}disk-{1:02d}'.format(instance_name, i + 1)
    config = {
            'name': name
        }

    new_disk = compute.disks().insert(project=target_project_id, zone=target_zone,
                                      sourceImage=source_disk_image, body=config).execute()
    new_disks.append((name, new_disk))
new_disks

[('es-cluster-disk-01',
  {u'id': u'6743525392042058698',
   u'insertTime': u'2017-03-23T07:35:17.655-07:00',
   u'kind': u'compute#operation',
   u'name': u'operation-1490279717064-54b66c926cd41-cb7769be-96374247',
   u'operationType': u'insert',
   u'progress': 0,
   u'selfLink': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f/operations/operation-1490279717064-54b66c926cd41-cb7769be-96374247',
   u'status': u'PENDING',
   u'targetId': u'8568716669835174858',
   u'targetLink': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f/disks/es-cluster-disk-01',
   u'user': u'XXXXXXXXXXXX-compute@developer.gserviceaccount.com',
   u'zone': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f'}),
 ('es-cluster-disk-02',
  {u'id': u'1674876474185638857',
   u'insertTime': u'2017-03-23T07:35:18.541-07:00',
   u'kind': u'compute#operation',
   u'name': u'operation-1490279717937-54b66c9341f69-9c2b142f-60f9f3d8',
   u'op

要求発行に成功したら、statusの確認を行います。全てREADYとなればOKです。

In [20]:
status = []
for name, disk in new_disks:
    result = compute.disks().get(project=target_project_id, zone=target_zone,
                                 disk=name).execute()
    status.append(result['status'])
status

[u'READY', u'READY', u'READY']

サイズの変更を行います。

In [21]:
results = []
for name, disk in new_disks:
    config = {
            'sizeGb': disk_size_gb
        }

    result = compute.disks().resize(project=target_project_id, zone=target_zone,
                                      disk=name,
                                      body=config).execute()
    results.append(result)
results

[{u'id': u'7416271034719822807',
  u'insertTime': u'2017-03-23T07:35:37.030-07:00',
  u'kind': u'compute#operation',
  u'name': u'operation-1490279736861-54b66ca54e148-f9fb49c4-e1b1e82b',
  u'operationType': u'resizeDisk',
  u'progress': 0,
  u'selfLink': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f/operations/operation-1490279736861-54b66ca54e148-f9fb49c4-e1b1e82b',
  u'status': u'PENDING',
  u'targetId': u'8568716669835174858',
  u'targetLink': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f/disks/es-cluster-disk-01',
  u'user': u'XXXXXXXXXXXX-compute@developer.gserviceaccount.com',
  u'zone': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f'},
 {u'id': u'5833448206003758038',
  u'insertTime': u'2017-03-23T07:35:37.539-07:00',
  u'kind': u'compute#operation',
  u'name': u'operation-1490279737352-54b66ca5c5f41-628c11ba-8a8aa256',
  u'operationType': u'resizeDisk',
  u'progress': 0,
  u'selfLink': 

ディスクサイズが変更されており、statusがREADYであることを確認します。

In [22]:
status = []
for name, disk in new_disks:
    result = compute.disks().get(project=target_project_id, zone=target_zone,
                                 disk=name).execute()
    status.append((result['status'], result['sizeGb']))
status

[(u'READY', u'1024'), (u'READY', u'1024'), (u'READY', u'1024')]

Configを指示してマシンを起動します。

In [23]:
new_vms = []
for i, disk in enumerate(new_disks):
    name = '{0}{1:02d}'.format(instance_name, i + 1)
    config = {
            'name': name,
            'machineType': "zones/{}/machineTypes/{}".format(target_zone, machine_type),
            'disks': [
                {
                    'boot': True,
                    'autoDelete': True,
                    'source': disk[1]['targetLink'],
                }
            ],
            'networkInterfaces': [{
                'network': 'global/networks/default',
                'accessConfigs': [
                    {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
                ]
            }],
            'serviceAccounts': [{
                'email': 'default',
                'scopes': [
                    'https://www.googleapis.com/auth/devstorage.read_write',
                    'https://www.googleapis.com/auth/logging.write'
                ]
            }],
            'metadata': {
                'items': []
            }
        }

    new_vm = compute.instances().insert(project=target_project_id, zone=target_zone, body=config).execute()
    new_vms.append((name, new_vm))
new_vms

[('es-cluster-01',
  {u'id': u'8250746789074723756',
   u'insertTime': u'2017-03-23T07:35:47.882-07:00',
   u'kind': u'compute#operation',
   u'name': u'operation-1490279747176-54b66caf24641-0400efe4-e25d2ffd',
   u'operationType': u'insert',
   u'progress': 0,
   u'selfLink': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f/operations/operation-1490279747176-54b66caf24641-0400efe4-e25d2ffd',
   u'status': u'PENDING',
   u'targetId': u'9129921960296211372',
   u'targetLink': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f/instances/es-cluster-01',
   u'user': u'XXXXXXXXXXXX-compute@developer.gserviceaccount.com',
   u'zone': u'https://www.googleapis.com/compute/v1/projects/testdev/zones/us-central1-f'}),
 ('es-cluster-02',
  {u'id': u'1509235558562292651',
   u'insertTime': u'2017-03-23T07:35:48.982-07:00',
   u'kind': u'compute#operation',
   u'name': u'operation-1490279748182-54b66cb019ff1-4e249edb-0878d0d6',
   u'operationType

statusがRUNNINGであることを確認する。

In [25]:
status = []
for name, instance in new_vms:
    s = compute.instances().get(project=target_project_id, zone=target_zone, instance=name).execute()
    status.append(s['status'])
status

[u'RUNNING', u'RUNNING', u'RUNNING']

対象マシンのIPアドレスを列挙する。

In [26]:
host_list = []
for name, instance in new_vms:
    s = compute.instances().get(project=target_project_id, zone=target_zone, instance=name).execute()
    host_list.append((name, s['networkInterfaces'][0]['networkIP'],
                      s['networkInterfaces'][0]['accessConfigs'][0]['natIP']))
host_list

[('es-cluster-01', u'10.128.0.2', u'130.211.181.251'),
 ('es-cluster-02', u'10.128.0.3', u'35.184.36.21'),
 ('es-cluster-03', u'10.128.0.4', u'35.184.148.62')]

pingが通ることを確認します。

In [27]:
import commands, sys
results = map(lambda host:commands.getoutput('ping -c 4 ' + host[2]), host_list)
results

['PING 130.211.181.251 (130.211.181.251): 56 data bytes\n64 bytes from 130.211.181.251: icmp_seq=0 ttl=55 time=127.655 ms\n64 bytes from 130.211.181.251: icmp_seq=1 ttl=55 time=127.055 ms\n64 bytes from 130.211.181.251: icmp_seq=2 ttl=55 time=127.052 ms\n64 bytes from 130.211.181.251: icmp_seq=3 ttl=55 time=127.051 ms\n--- 130.211.181.251 ping statistics ---\n4 packets transmitted, 4 packets received, 0% packet loss\nround-trip min/avg/max/stddev = 127.051/127.203/127.655/0.261 ms',
 'PING 35.184.36.21 (35.184.36.21): 56 data bytes\n64 bytes from 35.184.36.21: icmp_seq=0 ttl=55 time=128.380 ms\n64 bytes from 35.184.36.21: icmp_seq=1 ttl=55 time=127.199 ms\n64 bytes from 35.184.36.21: icmp_seq=2 ttl=55 time=127.167 ms\n64 bytes from 35.184.36.21: icmp_seq=3 ttl=55 time=127.166 ms\n--- 35.184.36.21 ping statistics ---\n4 packets transmitted, 4 packets received, 0% packet loss\nround-trip min/avg/max/stddev = 127.166/127.478/128.380/0.521 ms',
 'PING 35.184.148.62 (35.184.148.62): 56 data

Ansibleによる疎通確認も実施します。

In [28]:
import os
import tempfile
temp_dir = tempfile.mkdtemp()

with open(os.path.join(temp_dir, 'hosts'), 'w') as f:
    for name, priv, pub in host_list:
        f.write('{} ansible_host={} ansible_ssh_user=ansible ansible_ssh_private_key_file=~/.ssh/ansible_id_rsa\n'.format(name, pub))
!cat {temp_dir}/hosts

es-cluster-01 ansible_host=130.211.181.251 ansible_ssh_user=ansible ansible_ssh_private_key_file=~/.ssh/ansible_id_rsa
es-cluster-02 ansible_host=35.184.36.21 ansible_ssh_user=ansible ansible_ssh_private_key_file=~/.ssh/ansible_id_rsa
es-cluster-03 ansible_host=35.184.148.62 ansible_ssh_user=ansible ansible_ssh_private_key_file=~/.ssh/ansible_id_rsa


In [29]:
!ansible -m ping -i {temp_dir}/hosts all

/usr/local/lib/python2.7/dist-packages/cffi/model.py:525: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())
es-cluster-03 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
es-cluster-01 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
es-cluster-02 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


In [30]:
!ansible -a 'df -h' -i {temp_dir}/hosts all

/usr/local/lib/python2.7/dist-packages/cffi/model.py:525: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())
es-cluster-03 | SUCCESS | rc=0 >>
Filesystem      Size  Used Avail Use% Mounted on
/dev/sda1       1.0T  1.4G 1023G   1% /
devtmpfs         30G     0   30G   0% /dev
tmpfs            30G     0   30G   0% /dev/shm
tmpfs            30G  8.3M   30G   1% /run
tmpfs            30G     0   30G   0% /sys/fs/cgroup
tmpfs           5.9G     0  5.9G   0% /run/user/1000

es-cluster-01 | SUCCESS | rc=0 >>
Filesystem      Size  Used Avail Use% Mounted on
/dev/sda1       1.0T  1.4G 1023G   1% /
devtmpfs         30G     0   30G   0% /dev
tmpfs            30G     0   30G   0% /dev/shm
tmpfs            30G  8.3M   30G   1% /run
tmpfs            30G     0   30G   0% /sys/fs/cgroup
tmpfs           5.9G     0  5.9G   0% /run/user/1000

es-cluster-02 | SUCCESS | rc=0 >>
Filesystem      Size  Used Avail Use

全てのインスタンスに対しpingが通ればGCEインスタンス生成は完了です。

### 名前による疎通確認

GCEインスタンス中では、(内部的には)インスタンス名により名前解決することができる。このことを念のため確認しておく。

In [31]:
for name, instance in new_vms:
    !ansible -a 'ping -c 4 {name}' -i {temp_dir}/hosts all

/usr/local/lib/python2.7/dist-packages/cffi/model.py:525: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())
es-cluster-01 | SUCCESS | rc=0 >>
PING es-cluster-01.c.testdev.internal (XXX.XXX.XXX.2) 56(84) bytes of data.
64 bytes from es-cluster-01.c.testdev.internal (XXX.XXX.XXX.2): icmp_seq=1 ttl=64 time=0.059 ms
64 bytes from es-cluster-01.c.testdev.internal (XXX.XXX.XXX.2): icmp_seq=2 ttl=64 time=0.066 ms
64 bytes from es-cluster-01.c.testdev.internal (XXX.XXX.XXX.2): icmp_seq=3 ttl=64 time=0.081 ms
64 bytes from es-cluster-01.c.testdev.internal (XXX.XXX.XXX.2): icmp_seq=4 ttl=64 time=0.059 ms

--- es-cluster-01.c.testdev.internal ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 2999ms
rtt min/avg/max/mdev = 0.059/0.066/0.081/0.010 ms

es-cluster-03 | SUCCESS | rc=0 >>
PING es-cluster-01.c.testdev.internal (XXX.XXX.XXX.2) 56(84) bytes of data.
64 bytes from es-clu

## Inventory作成用のインスタンスリストを作成

ホストのパブリックDNS一覧をリスト化します  
次のブックの[Inventory作成](01_03_Install.ipynb#Inventory作成)で使用します。

In [32]:
host_list_pub = ['\'host{0}_name\':\'{1}\''.format(str(i+1), x[0]) for i,x in enumerate(host_list)] 
host_list_priv = ['\'host{0}_addr\':\'{1}\''.format(str(i+1), x[2]) for i,x in enumerate(host_list)] 
print('host_list = {{{0}}}'.format(','.join(host_list_pub + host_list_priv)))

host_list = {'host1_name':'es-cluster-01','host2_name':'es-cluster-02','host3_name':'es-cluster-03','host1_addr':'130.211.181.251','host2_addr':'35.184.36.21','host3_addr':'35.184.148.62'}
